# Trial code with stratified k fold

In [1]:
import tensorflow as tf
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout, GaussianNoise
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical 
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
df = pd.read_excel('Overall Colorectal Cancer Generated data.xlsx')
df.set_index('Name',inplace=True)

In [3]:
df_outputs= df['Marker']
df.drop('Marker',axis=1,inplace=True)
df_inputs=df

In [4]:
#splitting the data to training and testing
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(df_inputs, df_outputs, test_size=0.3, shuffle=True)

In [5]:
train_outputs = to_categorical(train_outputs)
test_outputs = to_categorical(test_outputs)

In [6]:
#Parameters used in the network
PIs = 0
number_of_glycans = len(df_inputs.columns) #it's actually the number of glycans
iterations = 500


In [7]:
def neural_network():
    model =tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(units = 24,
                             activation='sigmoid',
                             ))

    model.add(tf.keras.layers.Dense(units = 10,
                             activation='sigmoid'))

    model.add(tf.keras.layers.Dense(units = 2,
                             activation='softmax'))
    
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        loss = 'categorical_crossentropy',
        metrics = ['accuracy'])
    
    return model

    #model = Sequential([
        #GaussianNoise(0.1,
        #    input_shape= (number_of_genes + PIs,)),
        #Dense(units = 24,
              #activation= 'sigmoid',
              #input_shape= (number_of_glycans + PIs,)),
       # Dropout(0.2),
       # Dense(units = 10,
       #       activation='sigmoid'),
       # Dense(units = 2,
       #       activation = 'softmax')])

In [8]:
#model.compile(
    #optimizer = Adam(lr=0.01),
    #loss = 'categorical_crossentropy',
    #metrics = ['accuracy'])

In [9]:
model = neural_network()

In [10]:
model.fit(
    train_inputs.to_numpy(),
    train_outputs,
    epochs = iterations,
    batch_size = 32,
    verbose = 1,
    shuffle = False)

Train on 1433 samples
Epoch 1/500
1433/1433 [==============================] - 0s 336us/sample - loss: 0.6896 - accuracy: 0.5304
Epoch 2/500
1433/1433 [==============================] - 0s 29us/sample - loss: 0.6648 - accuracy: 0.6916
Epoch 3/500
1433/1433 [==============================] - 0s 29us/sample - loss: 0.6423 - accuracy: 0.7216
Epoch 4/500
1433/1433 [==============================] - 0s 31us/sample - loss: 0.6094 - accuracy: 0.7537
Epoch 5/500
1433/1433 [==============================] - 0s 29us/sample - loss: 0.5688 - accuracy: 0.7676
Epoch 6/500
1433/1433 [==============================] - 0s 30us/sample - loss: 0.5282 - accuracy: 0.7760
Epoch 7/500
1433/1433 [==============================] - 0s 29us/sample - loss: 0.4941 - accuracy: 0.7879
Epoch 8/500
1433/1433 [==============================] - 0s 32us/sample - loss: 0.4685 - accuracy: 0.7927
Epoch 9/500
1433/1433 [==============================] - 0s 33us/sample - loss: 0.4492 - accuracy: 0.8018
Epoch 10/500
1433/1433 

In [11]:
print(model.evaluate(
    test_inputs,
    test_outputs,
    batch_size = 32,
    verbose = 1))

615/615 [==============================] - 0s 249us/sample - loss: 0.1994 - accuracy: 0.9203
[0.19943678912108506, 0.9203252]


In [12]:
# Wrap Keras model so it can be used by scikit-learn
ANN = KerasClassifier(build_fn = neural_network,
                                 epochs=100,
                                 batch_size= 32,
                                 verbose = 1)

In [13]:
pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', ANN)])

In [18]:
c = cross_val_score(pipeline,
                    df_inputs.to_numpy(), df_outputs,
                    cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=101),
                    scoring='f1',
                    verbose=0)

Train on 1638 samples
Epoch 1/100
1638/1638 [==============================] - 1s 826us/sample - loss: 0.7366 - accuracy: 0.5055
Epoch 2/100
1638/1638 [==============================] - 0s 57us/sample - loss: 0.6269 - accuracy: 0.8126
Epoch 3/100
1638/1638 [==============================] - 0s 57us/sample - loss: 0.5718 - accuracy: 0.8816
Epoch 4/100
1638/1638 [==============================] - 0s 60us/sample - loss: 0.5027 - accuracy: 0.8877
Epoch 5/100
1638/1638 [==============================] - 0s 119us/sample - loss: 0.4269 - accuracy: 0.8987
Epoch 6/100
1638/1638 [==============================] - 0s 100us/sample - loss: 0.3592 - accuracy: 0.9048
Epoch 7/100
1638/1638 [==============================] - 0s 94us/sample - loss: 0.3086 - accuracy: 0.9090
Epoch 8/100
1638/1638 [==============================] - 0s 108us/sample - loss: 0.2715 - accuracy: 0.9127
Epoch 9/100
1638/1638 [==============================] - 0s 123us/sample - loss: 0.2456 - accuracy: 0.9206
Epoch 10/100
1638/1

KeyboardInterrupt: 

In [19]:
print(c)
print("Results: %.2f (%.2f)" % (c.mean(), c.std()))

[0.90452261 0.91803279 0.93366093 0.91725768 0.90537084]
Results: 0.92 (0.01)


In [16]:
#Calculate the labels for the test set
labels_test = model.predict(test_inputs)
labels_test = pd.DataFrame(data = labels_test)

In [17]:
#Calculate again the labels for the train set
labels_train = model.predict(train_inputs)
labels_train = pd.DataFrame(data = labels_train)